In [1]:
!export LC_ALL=en_US.UTF-8
!CUDA_VISIBLE_DEVICES="2"
import sys
import tensorflow as tf
import json
from pprint import pprint
import re
import numpy as np
from embedding import *
import nltk
import itertools
import random
np.random.seed(0)
orig_stdout = sys.stdout

In [2]:
#SpaCy
import spacy
nlp = spacy.load('en_core_web_lg')

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

['/gpu:0']

In [4]:
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.python.layers.core import Dense

In [5]:
data = json.load(open('../train-v1.1.json'))

In [6]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
nltkStopWords = stopwords.words('english')
import nltk

In [7]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [8]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [9]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [10]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [11]:
wordsToTake = 80000
words = findKMostFrequentWords(400000)

In [12]:
len(words)

102979

In [13]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


PAD_TOKEN = _add_word_reduced(PAD_WORD)
UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
    if(len(reduced_glove) == wordsToTake):
        break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(80000, 100)

In [14]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i], end = ' ', sep = ' ')

cuda2.cims.nyu.edu
1493
rev . john j. cavanaugh , c.s.c . served more than half , lobund institute for animal studies and medieval institute . hall of liberal arts ( 

In [15]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 'through', '

In [16]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(101, 104)

In [17]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1493
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [18]:
len(X_train_comp_all)

86106

In [19]:
examples_to_take_train = 86080

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [20]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [21]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

In [22]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)

(86080, 766)


In [23]:
print(reduced_glove.shape)

(80000, 100)


In [24]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(PAD_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
        if(j<max_len):
            X_indices[i,j] = look_up_word_reduced(END_WORD)
    return X_indices

In [25]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [26]:
document_tokens[0]

array([ 2039,    39,    11,   116,     6,    91, 15677,  1753,  6427,
         100,     4, 73713, 73714,     8, 37471, 21714, 73715,  3204,
         649,   571,     8, 17202,  2039,     9,    43,  3706,     5,
        1266,     7,  2039, 21017,    67,    35,  1750,    14,   120,
           9,    11, 33021,  9781,    21,  2039,     7,    63,  2039,
       21017,    28, 13701,  3204,     9,    40, 11991,     5,     8,
          28,  1021,    10,    38,   416,  2114,  2039,   415,    72,
         120,  1487,     7,     4,    48,     8,   180,   526,    82,
        2006,  1299,    14,   535,    21,  2039,    22, 17202, 37471,
       21714,   429,    12,   405, 73716,     7,  2039,    41,    39,
          11,  2863, 35780,     5,    16, 73717,    15,   472, 73718,
           7,     4,  2006,    39,    60,    11, 35780,     6,  2039,
          18,    92, 21062,    94,     8,    31,   169,  1031,     9,
          20, 41874,    78,    18,   191, 21062,    94,     7,     3,
           0,     0,

In [27]:
answer_labels.shape

(86080, 766)

In [28]:
import math

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    start = 0
    while start < len(inputs[0]):
        end = min(len(inputs[0]), start + batch_size)
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_mask': [],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[],
                 }
        
        for index,inp in enumerate(inputs):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            
            if index == 0:
                output['document_tokens'].append(inp[start:end,0:maxD])
            elif index==1:
                output['document_lengths'].append(inp[start:end])
            elif index==2:
                output['answer_labels'].append(inp[start:end,0:maxD])
            elif index==3:
                output['answer_mask'].append(inp[start:end,0:maxA,0:maxD])
            elif index==4:
                output['answer_lengths'].append(inp[start:end])
            elif index==5:
                output['question_input_tokens'].append(inp[start:end, 0:maxQ])
            elif index==6:
                output['question_output_tokens'].append(inp[start:end, 0:maxQ])
            elif index==7:
                output['question_lengths'].append(inp[start:end])
            
        output["document_tokens"] = np.array(output["document_tokens"])
        output["document_lengths"] = np.array(output["document_lengths"])
        output["answer_labels"] = np.array(output["answer_labels"])
        output["answer_mask"] = np.array(output["answer_mask"])
        output["answer_lengths"] = np.array(output["answer_lengths"])
        output["question_input_tokens"] = np.array(output["question_input_tokens"])
        output["question_output_tokens"] = np.array(output["question_output_tokens"])
        output["question_lengths"] = np.array(output["question_lengths"])
        outputs.append(output)
        start = start + batch_size
            
    return outputs


In [29]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths]
                    ,batch_size)

for b in batch_input:
    for k, v in b.items():
        b[k] = v.squeeze(0)

In [30]:
print("No of features:",len(batch_input[0]))
print("No of batches:",len(batch_input))

No of features: 8
No of batches: 2690


In [31]:
split_ratio = 0.999
split = int(len(batch_input) * split_ratio)
batch_input_train = batch_input[0:split]
batch_input_test = batch_input[split:]

if len(batch_input_train[-1]['document_tokens'] < batch_size):
    batch_input_train = batch_input_train[:-1]

if len(batch_input_test) > 0 and len(batch_input_test[-1]['document_tokens'] < batch_size):
    batch_input_test = batch_input_test[:-1]
    
print("Number of train batches = ", len(batch_input_train))
print("Number of test batches = ", len(batch_input_test))

Number of train batches =  2686
Number of test batches =  2


In [32]:
class HelperFunctions:
    def __init__(self):
        None
    def getDoc(self, batch, batch_num, example_num):
        document = itertools.takewhile(lambda t: t != END_TOKEN, batch[batch_num]['document_tokens'][example_num])
        doc = " ".join(look_up_token_reduced(token) for token in document)
        return doc

    def getQues(self, batch, batch_num, example_num):
        question = itertools.takewhile(lambda t: t != END_TOKEN, batch[batch_num]['question_output_tokens'][example_num])
        ques = " ".join(look_up_token_reduced(token) for token in question)
        return ques
    
    def getAns(self, batch, batch_num, example_num):
        ans = ''
        for i in range(batch[batch_num]['answer_lengths'][example_num]):
            ans = ans + look_up_token_reduced(batch[batch_num]['document_tokens'][example_num][np.where(batch[batch_num]['answer_mask'][example_num][i] == 1)[0][0]]) + ' '
        return ans
    
    def safe_exp(self, value):
        try:
            ans = math.exp(value)
        except OverflowError:
            ans = float("inf")
        return ans

In [33]:
class FileUtils:
    def __init__(self, filepath):
        self.filepath = filepath
    def clearFile(self):
        open(self.filepath, 'w').close()
    def appendString(self, string):
        with open(self.filepath, "a") as myfile:
            myfile.write(string)

In [34]:
class SimilarityScoreUtils:
    
    def __init__(self):
        None
    
    def bleu_score(self, ground_truth_question, generated_question):
        return nltk.translate.bleu_score.sentence_bleu(ground_truth_question,generated_question)
    
    def spacy_similarity_score(self, ground_truth_question, generated_question):
        return nlp(ground_truth_question).similarity(nlp(generated_question))
    
    def aggregate_score(self, inference, key):
        score = 0
        for s in inference[key]:
            score += s
        score /= len(inference[key])
        return score
    
    def find_similarity_score_from_inference(self, inferences, batch_size):
        bleu_score = 0
        spacy_score = 0
        for inference in inferences:
            bleu_score += self.aggregate_score(inference, 'bleu_score')
            spacy_score += self.aggregate_score(inference, 'spacy_score')
        bleu_score /= batch_size
        spacy_score /= batch_size
        return (bleu_score, spacy_score)

    def find_similarity_on_batch(self, batch, batch_size, qgen, beam = False, beam_num = 5):

        if beam :
            inferences = qgen.generateQuestionsFromBatchWithBeam([batch], 0, batch_size, beam_num, shuffle = False, generateGroundTruthQuestion = True)
        else:
            inferences = qgen.generateQuestionsFromBatch([batch], 0, batch_size, shuffle = False, generateGroundTruthQuestion = True)
            
        
        return self.find_similarity_score_from_inference(inferences, batch_size)
            
    def find_similarity_on_set(self, batch_input, batch_size, qgen, beam = False, beam_num = 5):
        bleu_scores = 0
        spacy_scores = 0
        num_batches = len(batch_input)
        
        for batch in batch_input:
            bleu_score, spacy_score = self.find_similarity_on_batch(batch, batch_size, qgen, beam, beam_num)
            bleu_scores += bleu_score
            spacy_scores += spacy_score
        
        bleu_scores /= num_batches
        spacy_scores /= num_batches

        return (bleu_scores, spacy_scores)

In [105]:
class InferenceUtils:
    def __init__(self, batch_size, qgen):
        self.batch_size = batch_size
        self.qgen = qgen
        
    def printInferences(self, inferences, printGroundTruthQuestion = True, printSimilarityScores = True):
        for inference in inferences:
            print('----------------------------------------------------------------------------------------------------')
            print('Comprehension : ')
            print(inference['passage'])

            if printGroundTruthQuestion:
                print('*****************************************************************************************************')
                print('Ground Truth Question : ')
                print(inference['ground_truth_question'])
            print('*****************************************************************************************************')
            print('Ground Truth Answer: ')
            print(inference['ground_truth_answer'])
            print('*****************************************************************************************************')
            for i,gen_ques in enumerate(inference['generated_questions']):
                print('Generated Question Number :  ', i+1)
                print(gen_ques)
                if printSimilarityScores:
                    print("BLEU Score : ", inference['bleu_score'][i])
                    print("SpaCy Similarity Score : ", inference['spacy_score'][i])
                print('*****************************************************************************************************')
            print('----------------------------------------------------------------------------------------------------')
        
    def createTokens(self, passage):
        passage = word_tokenize(passage.lower())
        passage_len = len(passage)
        p = np.array(sentences_to_indices_glove([passage], passage_len))
        p_len = np.array(passage_len).reshape((1,))
        p_batch = np.repeat(p,repeats=self.batch_size, axis = 0)
        p_len_batch = np.repeat(p_len,repeats=self.batch_size, axis = 0)
        return p_batch, p_len_batch
            
    
    def makeInferenceOnText(self, passage, answer = None, use_beam = False, num_questions = 5):
        
        
        if answer is None:
            answers = self.qgen.generateAnswers(passage)
            print("Answers = ", answers)
            answers = np.nonzero(answers)[0]
            outputs = []
            for i in np.split(answers, np.where(np.diff(answers) != 1)[0]+1):
                print(i)
                left,right = i[0],i[-1]+1
                answer = " ".join(word_tokenize(passage.lower())[j] for j in range(left,right))
                self.makeInferenceOnText(passage, answer, use_beam, num_questions)
            return
        
        passage = word_tokenize(passage.lower())
        passage_len = len(passage)

        answer = word_tokenize(answer.lower())
        answer_len = len(answer)    

        print("Passage Length = ", passage_len, ", Answer Length = ",answer_len)

        left,right = find_sub_list(answer,passage)
        if(left==-1):
            print("Couldn't find answer in the passage !!")
            return
        p = np.array(sentences_to_indices_glove([passage], passage_len))

        p_len = np.array(passage_len).reshape((1,))
        a_len = np.array(answer_len).reshape((1,))

        ans_labels = np.zeros((1,passage_len))
        ans_labels[0][left:right] = 1

        enc_mask = np.zeros((1,answer_len, passage_len))
        for i in range(left,right):
            enc_mask[0,i-left,i] = 1

        p_batch = np.repeat(p,repeats=self.batch_size, axis = 0)
        p_len_batch = np.repeat(p_len,repeats=self.batch_size, axis = 0)
        a_labels_batch = np.repeat(ans_labels,repeats=self.batch_size, axis = 0)
        enc_mask_batch = np.repeat(enc_mask,repeats=self.batch_size, axis = 0)
        a_len_batch = np.repeat(a_len,repeats=self.batch_size, axis = 0)
        
        output = {'document_tokens':p_batch,
                    'document_lengths':p_len_batch,
                    'answer_labels':a_labels_batch,
                    'answer_mask': enc_mask_batch,
                    'answer_lengths': a_len_batch
                 }
        if use_beam:
            inferences = self.qgen.generateQuestionsFromBatchWithBeam(
                batch = [output], 
                batchNum = 0, 
                numExamples = 1, 
                numQuestions = num_questions, 
                shuffle = False,
                generateGroundTruthQuestion = False,
                generateScores = False)
        else: 
            inferences = self.qgen.generateQuestionsFromBatch([output], 0, 1, shuffle = False, generateGroundTruthQuestion = False, generateScores = False)
            
        self.printInferences(inferences, printGroundTruthQuestion = False, printSimilarityScores = False)

In [84]:
class QGen:
    def __init__(
        self, learning_rate, cell_size , n_layers, reduced_glove, batch_size, 
            attention_type = 'Luong', cell_type = 'GRU', sess=tf.Session(), 
            helperFunctions = HelperFunctions(), fileUtils = FileUtils('/data/ra2630/tfLog64k'), similarityScoreUtils = SimilarityScoreUtils() ,
            grad_clip=1.0, beam_width=10, force_teaching_ratio=0.5, dropout_probability = [0.4, 0.3, 0.3]):
        self.learning_rate = learning_rate
        self.cell_size = cell_size
        self.cell_type = cell_type
        self.attention_type = attention_type
        self.n_layers = n_layers
        self.grad_clip = grad_clip
        self.glove_embedding = reduced_glove
        self.batch_size = batch_size
        self.beam_width = beam_width
        self.force_teaching_ratio = force_teaching_ratio
        self.dropout_probability_document_encoder = dropout_probability[0]
        self.dropout_probability_answer_encoder = dropout_probability[1]
        self.dropout_probability_question_decoder = dropout_probability[2]
        self.sess = sess
        self.helperFunctions = helperFunctions
        self.fileUtils = fileUtils
        self.similarityScoreUtils = similarityScoreUtils
        self.inferenceUtils = InferenceUtils(self.batch_size, self)
        self.build_graph()
        
    def saveSession(self, filepath):
        print("Saving Session to file : ", filepath)
        saver = tf.train.Saver()
        saver.save(self.sess, filepath)
        print("Session Saved !")
        
    def loadFromSession(self, filepath):
        print("Loading Session from file ", filepath)
        saver = tf.train.Saver()
        saver.restore(self.sess, filepath)
        self.sess.run(tf.tables_initializer())
        print("Session restored !")


    def build_graph(self):
        self.add_embedding_layer()
        self.add_document_encoder_layer()
        self.add_answer_encoder_layer()
        with tf.variable_scope('decode'):
            self.add_decoder_for_training()
        with tf.variable_scope('decode', reuse=True):
            self.add_beam_decoder_for_inference()
        self.add_decoder_for_inference()
            
        
        self.add_backward_path()


    def add_embedding_layer(self): 
        self.embedding = tf.get_variable("embedding", initializer=self.glove_embedding)
        self. embedding = tf.cast(self.embedding, dtype=tf.float32)
        self.embedding_dimensions = reduced_glove.shape[1]
        self.vocabulary_size = reduced_glove.shape[0]
        
    def add_document_encoder_layer(self):
        self.d_tokens = tf.placeholder(tf.int32, shape=[None, None])
        self.d_lengths = tf.placeholder(tf.int32, shape=[None])
        
        document_emb = tf.nn.embedding_lookup(self.embedding, self.d_tokens)
        document_emb = tf.cast(document_emb, dtype=tf.float64)
        
        if self.cell_type == 'LSTM':
            self.document_forward_cell = tf.contrib.rnn.LSTMCell(self.cell_size)
            self.document_backward_cell = tf.contrib.rnn.LSTMCell(self.cell_size)
        elif self.cell_type == 'GRU':
            self.document_forward_cell = tf.contrib.rnn.GRUCell(self.cell_size)
            self.document_backward_cell = tf.contrib.rnn.GRUCell(self.cell_size)
        
        self.document_forward_cell = tf.nn.rnn_cell.DropoutWrapper(
            self.document_forward_cell, 
            output_keep_prob = self.dropout_probability_document_encoder,
            state_keep_prob = self.dropout_probability_document_encoder
        )
        
        self.document_backward_cell = tf.nn.rnn_cell.DropoutWrapper(
            self.document_backward_cell, 
            output_keep_prob=self.dropout_probability_document_encoder,
            state_keep_prob = self.dropout_probability_document_encoder
        )
        
        self.document_encoder_outputs, self.document_encoder_states = tf.nn.bidirectional_dynamic_rnn(
            self.document_forward_cell, 
            self.document_backward_cell, 
            document_emb, 
            self.d_lengths, 
            dtype=tf.float64
        )
        
        self.document_encoder_outputs = tf.concat(self.document_encoder_outputs, 2)
        self.document_encoder_outputs = tf.cast(self.document_encoder_outputs,tf.float32)
        self.answer_tags = tf.layers.dense(inputs = self.document_encoder_outputs, units=2)


        self.a_labels = tf.placeholder(tf.int32, shape=[None, None])
        self.answer_mask = tf.sequence_mask(self.d_lengths, dtype=tf.float32)
        self.answer_loss = seq2seq.sequence_loss(
            logits=self.answer_tags, targets=self.a_labels, weights=self.answer_mask)
        
        self.answer_loss = tf.Print(self.answer_loss, [self.answer_loss], message="This is answer_loss: ")
        


    def add_answer_encoder_layer(self):
        self.answer_encoder_input_mask = tf.placeholder(
        tf.float32, shape=[None, None, None])
        
        self.answer_encoder_inputs = tf.matmul(self.answer_encoder_input_mask, self.document_encoder_outputs)
        self.answer_encoder_lengths = tf.placeholder(tf.int32, shape=[None])

        if self.cell_type == 'GRU':
            self.answer_encoder_cell = tf.contrib.rnn.GRUCell(self.document_forward_cell.state_size + self.document_backward_cell.state_size)
            self.question_decoder_cell = tf.contrib.rnn.GRUCell(self.answer_encoder_cell.state_size)
        
        elif self.cell_type == 'LSTM':
            self.answer_encoder_cell = tf.contrib.rnn.LSTMCell(self.document_forward_cell.state_size[0] + self.document_backward_cell.state_size[0])
            self.question_decoder_cell = tf.contrib.rnn.LSTMCell(self.answer_encoder_cell.state_size[0])
            
        self.answer_encoder_cell = tf.nn.rnn_cell.DropoutWrapper(
            self.answer_encoder_cell,
            state_keep_prob=self.dropout_probability_answer_encoder
        )
        
        self.question_decoder_cell = tf.nn.rnn_cell.DropoutWrapper(
            self.question_decoder_cell, 
            state_keep_prob = self.dropout_probability_question_decoder,
            output_keep_prob=self.dropout_probability_question_decoder
        )
            
        _, self.answer_encoder_state = tf.nn.dynamic_rnn(
            self.answer_encoder_cell, self.answer_encoder_inputs, self.answer_encoder_lengths, dtype=tf.float32)



    def add_attention_for_training(self):
        if self.attention_type == 'Luong':
            self.attention_function = tf.contrib.seq2seq.LuongAttention
        else:
            self.attention_function = tf.contrib.seq2seq.BahdanauAttention
            
        if self.cell_type == 'GRU':
            attention_mechanism = self.attention_function(
                num_units = self.answer_encoder_cell.state_size, 
                memory = self.document_encoder_outputs,
                memory_sequence_length = self.d_lengths)

            self.question_decoder_cell_attention = tf.contrib.seq2seq.AttentionWrapper(
                cell = self.question_decoder_cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = self.question_decoder_cell.state_size)

        elif self.cell_type == 'LSTM':
            attention_mechanism = self.attention_function(
                num_units = self.answer_encoder_cell.state_size[0], 
                memory = self.document_encoder_outputs,
                memory_sequence_length = self.d_lengths)

            self.question_decoder_cell_attention = tf.contrib.seq2seq.AttentionWrapper(
                cell = self.question_decoder_cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = self.question_decoder_cell.state_size[0])
        
        self.question_decoder_cell_attention_without_beam = self.question_decoder_cell_attention


    def add_decoder_for_training(self):
        self.add_attention_for_training()
        self.question_decoder_inputs = tf.placeholder(tf.int32, shape=[None, None])
        self.question_decoder_lengths = tf.placeholder(tf.int32, shape=[None])

        question_decoder_embedding = tf.nn.embedding_lookup(self.embedding, self.question_decoder_inputs)
        question_decoder_embedding = tf.cast(question_decoder_embedding,tf.float32)

        #helper = seq2seq.TrainingHelper(question_decoder_embedding , self.question_decoder_lengths)
        helper = seq2seq.ScheduledEmbeddingTrainingHelper(
            inputs = question_decoder_embedding,
            sequence_length = self.question_decoder_lengths,
            embedding = self.embedding,
            sampling_probability = 1 - self.force_teaching_ratio,
            time_major = False)

        self.projection = Dense(self.vocabulary_size, use_bias=False)
        
        decoder = seq2seq.BasicDecoder(
            self.question_decoder_cell_attention, 
            helper, 
            self.question_decoder_cell_attention.zero_state(self.batch_size, dtype=tf.float32).clone(
              cell_state=self.answer_encoder_state),
            output_layer = self.projection
        )
        
        decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder)
        self.training_question_decoder_outputs = decoder_outputs.rnn_output
        
    def add_decoder_for_inference(self):
        helper = seq2seq.GreedyEmbeddingHelper(
            self.embedding, 
            tf.fill([self.batch_size], START_TOKEN), 
            END_TOKEN
        )
        decoder = seq2seq.BasicDecoder(
            self.question_decoder_cell_attention_without_beam, 
            helper, 
            self.question_decoder_cell_attention_without_beam.zero_state(self.batch_size, dtype=tf.float32).clone(
                  cell_state=self.answer_encoder_state), output_layer=self.projection)
        
        decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
        self.decoder_outputs = decoder_outputs.rnn_output
        
        


    def add_beam_attention_for_inference(self):
        self.document_encoder_output_tiled = tf.contrib.seq2seq.tile_batch(self.document_encoder_outputs, self.beam_width)
        self.answer_encoder_state_tiled = tf.contrib.seq2seq.tile_batch(self.answer_encoder_state, self.beam_width)
        self.document_lengths_tiled = tf.contrib.seq2seq.tile_batch(self.d_lengths, self.beam_width)

        if self.cell_type == 'GRU':
            attention_mechanism = self.attention_function(
                num_units = self.answer_encoder_cell.state_size, 
                memory = self.document_encoder_output_tiled,
                memory_sequence_length = self.document_lengths_tiled)

            self.question_decoder_cell_attention = tf.contrib.seq2seq.AttentionWrapper(
                cell = self.question_decoder_cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = self.question_decoder_cell.state_size)
            

        elif self.cell_type == 'LSTM':
            attention_mechanism = self.attention_function(
                num_units = self.answer_encoder_cell.state_size[0], 
                memory = self.document_encoder_output_tiled,
                memory_sequence_length = self.document_lengths_tiled)

            self.question_decoder_cell_attention = tf.contrib.seq2seq.AttentionWrapper(
                cell = self.question_decoder_cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = self.question_decoder_cell.state_size[0])



    def add_beam_decoder_for_inference(self):
        self.add_beam_attention_for_inference()
        
        decoder = tf.contrib.seq2seq.BeamSearchDecoder(
            cell = self.question_decoder_cell_attention,
            embedding = self.embedding,
            start_tokens = tf.fill([self.batch_size], START_TOKEN),
            end_token = END_TOKEN,
            initial_state = self.question_decoder_cell_attention.zero_state(self.batch_size * self.beam_width,tf.float32).clone(
                cell_state = self.answer_encoder_state_tiled),
            beam_width = self.beam_width,
            output_layer = self.projection,
            length_penalty_weight = 0.0)
        
        
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
            decoder = decoder,
            maximum_iterations = max_question_len
        )
        self.predicting_question_ids = predicting_decoder_output.predicted_ids[:, :, :]


    def add_backward_path(self):
        self.decoder_labels = tf.placeholder(tf.int64, shape=[None, None])
        question_mask = tf.sequence_mask(self.question_decoder_lengths ,dtype=tf.float32)
        self.question_loss = seq2seq.sequence_loss(
            logits = self.training_question_decoder_outputs, 
            targets = self.decoder_labels,
            weights = question_mask)
        self.question_loss = tf.Print(self.question_loss, [self.question_loss], message="This is question_loss: ")
        
        self.net_loss = self.question_loss + self.answer_loss
        
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        gvs = optimizer.compute_gradients(self.net_loss)
        capped_gvs = [(tf.clip_by_value(grad, -self.grad_clip, self.grad_clip), var) for grad, var in gvs]
        self.train_op = optimizer.apply_gradients(capped_gvs)


    def fit(self, batch_input_train, batch_input_test, path_to_save_file,save_after_every_epoch = 5, n_epoch=60, generateTrain = False, generateTest = False, clearFile = False, resumeTraining = False):

        if not resumeTraining:
            self.sess.run(tf.global_variables_initializer())
        if clearFile:
            self.fileUtils.clearFile()
        
        for epoch in range(1, n_epoch + 1):
            batch_loss = 0
            perplexity_score = 0
            for batchNum in range(len(batch_input_train)):
                t = self.sess.run([self.train_op, self.net_loss, self.question_loss, self.answer_loss], {
                    self.d_tokens: batch_input_train[batchNum]['document_tokens'],
                    self.d_lengths: batch_input_train[batchNum]['document_lengths'],
                    self.a_labels: batch_input_train[batchNum]['answer_labels'],
                    self.answer_encoder_input_mask: batch_input_train[batchNum]['answer_mask'],
                    self.answer_encoder_lengths: batch_input_train[batchNum]['answer_lengths'],
                    self.question_decoder_inputs: batch_input_train[batchNum]['question_input_tokens'],
                    self.decoder_labels: batch_input_train[batchNum]['question_output_tokens'],
                    self.question_decoder_lengths: batch_input_train[batchNum]['question_lengths']
                })
                p_score = self.helperFunctions.safe_exp((t[1] / max_question_len) * self.batch_size)
                print("Epoch : {0}, Batch: {1}/{2}, Loss: {3}, perplexity_score: {4}".format(epoch, batchNum, len(batch_input_train), t[1], p_score))
                batch_loss += t[1]
                perplexity_score+= p_score
                sys.stdout.flush()
            batch_loss /= len(batch_input_train)
            print("Epochs: {0}/{1} | Batch - Loss (Train): {2} | Perplexity (Train) : {3}".format(epoch, n_epoch, batch_loss, perplexity_score))
            sys.stdout.flush()
            
            
            
            batch_loss = 0
            perplexity_score = 0
            for batchNum in range(len(batch_input_test)):
                t = self.sess.run([self.net_loss], {
                    self.d_tokens: batch_input_test[batchNum]['document_tokens'],
                    self.d_lengths: batch_input_test[batchNum]['document_lengths'],
                    self.a_labels: batch_input_test[batchNum]['answer_labels'],
                    self.answer_encoder_input_mask: batch_input_test[batchNum]['answer_mask'],
                    self.answer_encoder_lengths: batch_input_test[batchNum]['answer_lengths'],
                    self.question_decoder_inputs: batch_input_test[batchNum]['question_input_tokens'],
                    self.decoder_labels: batch_input_test[batchNum]['question_output_tokens'],
                    self.question_decoder_lengths: batch_input_test[batchNum]['question_lengths']
                })

                batch_loss += t[0]
                p_score = self.helperFunctions.safe_exp((t[0] / max_question_len) * self.batch_size)
                perplexity_score+= p_score

            batch_loss /= len(batch_input_test)

            print("Epochs: {0}/{1} | Batch - Loss (Test): {2} | Perplexity (Test) : {3}".format(epoch, n_epoch, batch_loss, perplexity_score))
            sys.stdout.flush()
                    
            
            if generateTrain:
                print("Train Samples Generated : ")
                inferences = self.generateQuestionsFromBatch(batch_input_train, random.randint(0,len(batch_input_train)-1), 5)
                self.inferenceUtils.printInferences(inferences)
                bleu_score, spacy_score = self.similarityScoreUtils.find_similarity_on_batch(
                    batch_input_train[random.randint(0, len(batch_input_train)-1)], self.batch_size, self, beam = False)
                print("For Non-Beam Question Generation, Bleu Score = {0} and Spacy Score = {1}".format(bleu_score, spacy_score))
                
                inferences = self.generateQuestionsFromBatchWithBeam(batch_input_train, random.randint(0,len(batch_input_train)-1), 5, 5)
                self.inferenceUtils.printInferences(inferences)
                bleu_score, spacy_score = self.similarityScoreUtils.find_similarity_on_batch(
                    batch_input_train[random.randint(0, len(batch_input_train)-1)], self.batch_size, self, beam = True, beam_num = 5)
                print("For Beam Question Generation, Bleu Score = {0} and Spacy Score = {1}".format(bleu_score, spacy_score))
                
            if generateTest:
                print("Test Samples Generated : ")
                inferences = self.generateQuestionsFromBatch(batch_input_test, random.randint(0,len(batch_input_test)-1), 5)
                self.inferenceUtils.printInferences(inferences)
                bleu_score, spacy_score = self.similarityScoreUtils.find_similarity_on_batch(
                    batch_input_test[random.randint(0, len(batch_input_test)-1)], self.batch_size, self, beam = False)
                print("For Non-Beam Question Generation, Bleu Score = {0} and Spacy Score = {1}".format(bleu_score, spacy_score))
                
                inferences = self.generateQuestionsFromBatchWithBeam(batch_input_test, random.randint(0,len(batch_input_test)-1), 5, 5)
                self.inferenceUtils.printInferences(inferences)
                bleu_score, spacy_score = self.similarityScoreUtils.find_similarity_on_batch(
                    batch_input_test[random.randint(0, len(batch_input_test)-1)], self.batch_size, self, beam = True, beam_num = 5)
                print("For Beam Question Generation, Bleu Score = {0} and Spacy Score = {1}".format(bleu_score, spacy_score))
                
            if epoch % save_after_every_epoch == 0:
                self.saveSession(path_to_save_file)
    
            
    def generateAnswers(self, passage):
        p_batch, p_len_batch = self.inferenceUtils.createTokens(passage)
        answers = self.sess.run(qgen.answer_tags, {
            qgen.d_tokens: p_batch,
            qgen.d_lengths: p_len_batch,
        })
        answers = np.argmax(answers[0], 1)
        return answers
        
        
    def generateQuestionsFromBatch(self, batch, batchNum, numExamples, shuffle = True, generateGroundTruthQuestion = True, generateScores = True):
        
        inferences = []
        
        if shuffle:
            p = random.sample(range(0, self.batch_size), self.batch_size)
        else:
            p = range(0,self.batch_size)
        j = 0
        questions = self.sess.run(self.decoder_outputs, {
            self.d_tokens: batch[batchNum]['document_tokens'][p],
            self.d_lengths: batch[batchNum]['document_lengths'][p],
            self.a_labels: batch[batchNum]['answer_labels'][p],
            self.answer_encoder_input_mask: batch[batchNum]['answer_mask'][p],
            self.answer_encoder_lengths: batch[batchNum]['answer_lengths'][p],
        })
        p = p[:numExamples]

        qs = np.argmax(questions, 2)
        for i in p:
            inference = {
                'passage' : '',
                'ground_truth_question' : '',
                'ground_truth_answer' : '',
                'generated_questions' : [],
                'bleu_score' : [],
                'spacy_score' : []
            }
            question = itertools.takewhile(lambda t: t != END_TOKEN, qs[j])
            inference['passage'] = self.helperFunctions.getDoc(batch, batchNum , i)
            if generateGroundTruthQuestion:
                inference['ground_truth_question'] = self.helperFunctions.getQues(batch, batchNum, i)
            inference['ground_truth_answer'] = self.helperFunctions.getAns(batch, batchNum, i)
            generated_question = " ".join(look_up_token_reduced(token) for token in question)
            inference['generated_questions'] = [generated_question]
            if generateScores :
                inference['bleu_score'] = [self.similarityScoreUtils.bleu_score(inference['ground_truth_question'], generated_question)]
                inference['spacy_score'] = [self.similarityScoreUtils.spacy_similarity_score(inference['ground_truth_question'], generated_question)]
            j=j+1
            inferences.append(inference)

        return inferences
        
        
    
    def generateQuestionsFromBatchWithBeam(self, batch, batchNum, numExamples, numQuestions, shuffle = True, generateGroundTruthQuestion = True, generateScores = True):
        
        inferences = []
        if shuffle:
            p = random.sample(range(0, self.batch_size), self.batch_size)
        else:
            p = range(0,self.batch_size)
        j = 0
        questions = self.sess.run(self.predicting_question_ids, {
            self.d_tokens: batch[batchNum]['document_tokens'][p],
            self.d_lengths: batch[batchNum]['document_lengths'][p],
            self.a_labels: batch[batchNum]['answer_labels'][p],
            self.answer_encoder_input_mask: batch[batchNum]['answer_mask'][p],
            self.answer_encoder_lengths: batch[batchNum]['answer_lengths'][p],
        })
        p = p[:numExamples]
        for i in p:
            inference = {
                'passage' : '',
                'ground_truth_question' : '',
                'ground_truth_answer' : '',
                'generated_questions' : [],
                'bleu_score' : [],
                'spacy_score' : []
            }
            inference['passage'] = self.helperFunctions.getDoc(batch, batchNum ,i)
            if generateGroundTruthQuestion:
                inference['ground_truth_question'] = self.helperFunctions.getQues(batch, batchNum, i)
            inference['ground_truth_answer'] = self.helperFunctions.getAns(batch, batchNum, i)
            generated_questions = []
            bleu_scores = []
            spacy_scores = []
            for k in range(numQuestions):
                question = itertools.takewhile(lambda t: t != END_TOKEN, questions[j,:,k])
                generated_question = " ".join(look_up_token_reduced(token) for token in question)
                generated_questions.append(generated_question)
                if generateScores :
                    bleu_scores.append(self.similarityScoreUtils.bleu_score(inference['ground_truth_question'], generated_question))
                    spacy_scores.append(self.similarityScoreUtils.spacy_similarity_score(inference['ground_truth_question'], generated_question))
                
            j=j+1
            if generateScores :
                inference['bleu_score'] = bleu_scores
                inference['spacy_score'] = spacy_scores

            inference['generated_questions'] = generated_questions
            inferences.append(inference)
        return inferences

In [86]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

qgen = QGen(learning_rate = 3e-4,
            cell_size = 500,
            n_layers = 1,
            reduced_glove = reduced_glove,
            batch_size = batch_size,
            attention_type = 'Bahdanau',
            cell_type = 'LSTM',
            sess=session,
            grad_clip=1.,
            beam_width=10,
            force_teaching_ratio=0.5,
            dropout_probability = [1.0, 1.0, 1.0])

/data/ra2630/miniconda3/envs/qgen/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
sys.stdout = open('/data/ra2630/tfLog72k-4', 'a', 1)
qgen.fit(
    batch_input_train, 
    batch_input_test, 
    n_epoch=15,
    path_to_save_file = '/data/ra2630/tf72k-4',
    save_after_every_epoch = 3,
    generateTrain = True, 
    generateTest = True,
    clearFile=False, 
    resumeTraining = True)

In [87]:
qgen.loadFromSession("/data/ra2630/tf72k-2")

Loading Session from file  /data/ra2630/tf72k-2
INFO:tensorflow:Restoring parameters from /data/ra2630/tf72k-2
Session restored !


In [106]:
inferenceUtils = InferenceUtils(batch_size=batch_size, qgen=qgen)

In [ ]:
inferenceUtils.printInferences(qgen.generateQuestionsFromBatch(
    batch = batch_input_train, 
    batchNum = 2, 
    numExamples = 5, 
    shuffle = True
))

In [ ]:
inferenceUtils.printInferences(qgen.generateQuestionsFromBatchWithBeam(
            batch = batch_input_test, 
            batchNum = 0, 
            numExamples = 32, 
            numQuestions = 10, 
            shuffle = True,
            generateGroundTruthQuestion = True,
            generateScores = True
))

In [99]:
inferenceUtils.makeInferenceOnText(
    passage = "India, officially the Republic of India (IAST: Bhārat Gaṇarājya),[e] is a country in South Asia. It is the seventh-largest country by area, the second-most populous country (with over 1.2 billion people), and the most populous democracy in the world",
    answer = '1.2 billion people',
    num_questions = 10,
    use_beam = True)


Passage Length =  52 , Answer Length =  3
NumQues =  10
----------------------------------------------------------------------------------------------------
Comprehension : 
india , officially the republic of india ( iast : <UNK> <UNK> ) , [ e ] is a country in south asia . it is the seventh-largest country by area , the second-most populous country ( with over 1.2 billion people ) , and the most populous democracy in the world
*****************************************************************************************************
Ground Truth Answer: 
1.2 billion people 
*****************************************************************************************************
Generated Question Number :   1
how many people live in india ?
*****************************************************************************************************
Generated Question Number :   2
how many people does india have ?
******************************************************************************************

In [112]:
inferenceUtils.makeInferenceOnText(
    passage = "New York University (NYU) is a private nonprofit research university based in New York City. Founded in 1831, NYU's main campus is centered in Manhattan, located with its core in Greenwich Village, and campuses based throughout New York City. NYU is also a worldwide university, operating NYU Abu Dhabi and NYU Shanghai, and centers in Accra, Berlin, Buenos Aires, Florence, London, Madrid, Paris, Prague, Sydney, Tel Aviv, and Washington, D.C." , 
    use_beam = True, 
    num_questions = 10)

Answers =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[20 21]
Passage Length =  92 , Answer Length =  2
----------------------------------------------------------------------------------------------------
Comprehension : 
new york university ( nyu ) is a private nonprofit research university based in new york city . founded in 1831 , nyu 's main campus is centered in manhattan , located with its core in greenwich village , and campuses based throughout new york city . nyu is also a worldwide university , operating nyu abu dhabi and nyu shanghai , and centers in accra , berlin , buenos aires , florence , london , madrid , paris , prague , sydney , tel aviv , and washington , d.c .
*****************************************************************************************************
Ground Truth Answer: 
1831 , 
**********************************

----------------------------------------------------------------------------------------------------
Comprehension : 
new york university ( nyu ) is a private nonprofit research university based in new york city . founded in 1831 , nyu 's main campus is centered in manhattan , located with its core in greenwich village , and campuses based throughout new york city . nyu is also a worldwide university , operating nyu abu dhabi and nyu shanghai , and centers in accra , berlin , buenos aires , florence , london , madrid , paris , prague , sydney , tel aviv , and washington , d.c .
*****************************************************************************************************
Ground Truth Answer: 
university , operating nyu 
*****************************************************************************************************
Generated Question Number :   1
what is the name of the new york university ?
***********************************************************************************

In [113]:
inferenceUtils.makeInferenceOnText(
    passage = "New York University (NYU) is a private nonprofit research university based in New York City. Founded in 1831, NYU's main campus is centered in Manhattan, located with its core in Greenwich Village, and campuses based throughout New York City. NYU is also a worldwide university, operating NYU Abu Dhabi and NYU Shanghai, and centers in Accra, Berlin, Buenos Aires, Florence, London, Madrid, Paris, Prague, Sydney, Tel Aviv, and Washington, D.C." , 
    answer = 'private nonprofit research university' ,
    use_beam = True)

Passage Length =  92 , Answer Length =  4
----------------------------------------------------------------------------------------------------
Comprehension : 
new york university ( nyu ) is a private nonprofit research university based in new york city . founded in 1831 , nyu 's main campus is centered in manhattan , located with its core in greenwich village , and campuses based throughout new york city . nyu is also a worldwide university , operating nyu abu dhabi and nyu shanghai , and centers in accra , berlin , buenos aires , florence , london , madrid , paris , prague , sydney , tel aviv , and washington , d.c .
*****************************************************************************************************
Ground Truth Answer: 
private nonprofit research university 
*****************************************************************************************************
Generated Question Number :   1
what type of university is new york university ?
***************************

In [114]:
inferenceUtils.makeInferenceOnText(
    passage = 'The election of President and Vice President of the United States is an indirect election in which citizens of the United States who are registered to vote in one of the 50 U.S. states or in Washington, D.C. cast ballots not directly for those offices, but instead for members of the U.S. Electoral College, known as electors.' , 
    num_questions = 10 , 
    use_beam = True)

Answers =  [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[13]
Passage Length =  61 , Answer Length =  1
----------------------------------------------------------------------------------------------------
Comprehension : 
the election of president and vice president of the united states is an indirect election in which citizens of the united states who are registered to vote in one of the 50 u.s. states or in washington , d.c. cast ballots not directly for those offices , but instead for members of the u.s. electoral college , known as electors .
*****************************************************************************************************
Ground Truth Answer: 
indirect 
*****************************************************************************************************
Generated Question Number :   1
what type of election is the united states ?
*******************************************************

In [119]:
inferenceUtils.makeInferenceOnText(
    passage = 'The election of President and Vice President of the United States is an indirect election in which citizens of the United States who are registered to vote in one of the 50 U.S. states or in Washington, D.C. cast ballots not directly for those offices, but instead for members of the U.S. Electoral College, known as electors.' , 
    num_questions = 10 , 
    answer = 'electors' ,
    use_beam = True)

Passage Length =  61 , Answer Length =  1
----------------------------------------------------------------------------------------------------
Comprehension : 
the election of president and vice president of the united states is an indirect election in which citizens of the united states who are registered to vote in one of the 50 u.s. states or in washington , d.c. cast ballots not directly for those offices , but instead for members of the u.s. electoral college , known as electors .
*****************************************************************************************************
Ground Truth Answer: 
electors 
*****************************************************************************************************
Generated Question Number :   1
what is the term of the united states ?
*****************************************************************************************************
Generated Question Number :   2
what is the term for the united states ?
**************************

In [120]:
inferenceUtils.makeInferenceOnText(
    passage = 'The election of President and Vice President of the United States is an indirect election in which citizens of the United States who are registered to vote in one of the 50 U.S. states or in Washington, D.C. cast ballots not directly for those offices, but instead for members of the U.S. Electoral College, known as electors.' , 
    num_questions = 10 , 
    answer = '50' ,
    use_beam = True)

Passage Length =  61 , Answer Length =  1
----------------------------------------------------------------------------------------------------
Comprehension : 
the election of president and vice president of the united states is an indirect election in which citizens of the united states who are registered to vote in one of the 50 u.s. states or in washington , d.c. cast ballots not directly for those offices , but instead for members of the u.s. electoral college , known as electors .
*****************************************************************************************************
Ground Truth Answer: 
50 
*****************************************************************************************************
Generated Question Number :   1
how many us states are there ?
*****************************************************************************************************
Generated Question Number :   2
how many us states are in the united states ?
************************************

In [121]:
inferenceUtils.makeInferenceOnText(
    passage = 'the largest of these is the eldon square shopping centre , one of the largest city centre shopping complexes in the uk.', 
    num_questions = 10 , 
    use_beam = True)


Answers =  [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[6 7 8]
Passage Length =  23 , Answer Length =  3
----------------------------------------------------------------------------------------------------
Comprehension : 
the largest of these is the <UNK> square shopping centre , one of the largest city centre shopping complexes in the uk .
*****************************************************************************************************
Ground Truth Answer: 
<UNK> square shopping 
*****************************************************************************************************
Generated Question Number :   1
what is the largest city centre shopping center ?
*****************************************************************************************************
Generated Question Number :   2
what is the largest city centre shopping center ?
*****************************************************************************************************
Generated Question Number :   3
wha

In [128]:
inferenceUtils.makeInferenceOnText(
    passage = 'Sam Bowman is one of the leading researchers in the field of natural language processing (NLP), and recently joined NYU as an Assistant Professor in Computational Linguistics, a joint position between NYU’s Linguistics department, and the Center for Data Science.', 
    num_questions = 10 , 
    use_beam = True)


Answers =  [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
[0 1 2]
Passage Length =  48 , Answer Length =  3
----------------------------------------------------------------------------------------------------
Comprehension : 
sam <UNK> is one of the leading researchers in the field of natural language processing ( <UNK> ) , and recently joined nyu as an assistant professor in computational linguistics , a joint position between nyu ’ s linguistics department , and the center for data science .
*****************************************************************************************************
Ground Truth Answer: 
sam <UNK> is 
*****************************************************************************************************
Generated Question Number :   1
who is one of the leading researchers in the field of natural language processing ?
*********************************************************************************************

In [158]:
inferenceUtils.makeInferenceOnText(
    passage = 'Artificial neural networks are machine learning models : computational systems that are designed to learn how to perform some kind of behavior, given previous examples shown through data.  This can include anything from labeling an image, to translating a text, to steering a robot.', 
    num_questions = 5 , 
    use_beam = True)


Answers =  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1]
[0 1]
Passage Length =  49 , Answer Length =  2
----------------------------------------------------------------------------------------------------
Comprehension : 
artificial neural networks are machine learning models : computational systems that are designed to learn how to perform some kind of behavior , given previous examples shown through data . this can include anything from labeling an image , to translating a text , to steering a robot .
*****************************************************************************************************
Ground Truth Answer: 
artificial neural 
*****************************************************************************************************
Generated Question Number :   1
what type of learning are machine learning ?
*****************************************************************************************************
Generated Q

In [204]:
inferenceUtils.makeInferenceOnText(
    passage = "Chris, however, revealed he was told RCB would retain him, only to not hear from the franchise again. 'I was their biggest draw. It was disappointing from that end, because they had called me. They wanted me in the team and I was told that I will be retained,' Gayle told The Times of India. 'But they never called back after that. So that gave me the impression that they didn't want me, and it's fine.'", 
    answer = 'Chris',
    num_questions = 5 , 
    use_beam = True)






Passage Length =  92 , Answer Length =  1
----------------------------------------------------------------------------------------------------
Comprehension : 
chris , however , revealed he was told <UNK> would retain him , only to not hear from the franchise again . 'i was their biggest draw . it was disappointing from that end , because they had called me . they wanted me in the team and i was told that i will be retained , ' <UNK> told the times of india . <UNK> they never called back after that . so that gave me the impression that they did n't want me , and it 's fine . '
*****************************************************************************************************
Ground Truth Answer: 
chris 
*****************************************************************************************************
Generated Question Number :   1
who said that they would not be acted the times of india ?
********************************************************************************************

In [233]:
inferenceUtils.makeInferenceOnText("the american idol songwriter contest was also held this season . from ten of the most popular submissions , each of the final two contestants chose a song to perform , although neither of their selections was used as the `` coronation song '' . the winning song , `` the time of my life '' , was recorded by david cook and released on may 22 , 2008 . ",
    num_questions = 10 , 
    use_beam = True)

Answers =  [1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
[0 1 2 3 4 5]
Passage Length =  70 , Answer Length =  6
----------------------------------------------------------------------------------------------------
Comprehension : 
the american idol songwriter contest was also held this season . from ten of the most popular submissions , each of the final two contestants chose a song to perform , although neither of their selections was used as the `` coronation song '' . the winning song , `` the time of my life '' , was recorded by david cook and released on may 22 , 2008 .
*****************************************************************************************************
Ground Truth Answer: 
the american idol songwriter contest was 
*****************************************************************************************************
Generated Question Number :   1
did the idol of my li

In [40]:
sys.stdout = orig_stdout

In [232]:
10

10